In [ ]:
# Importing Libraries
import h5py
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2

In [ ]:
base_directory = os.path.join(os.getcwd(), "typhon_images", "image")
h5_file_paths = []

# Target size for resizing
# desired_size = (256, 256)
desired_size = (300, 300)

# collecting the .h5 files
for i in range(197901, 197902):
    target_path = os.path.join(base_directory, str(i))
    FILE_PATH = Path(target_path)          # converting the target path into a path object
    for file in FILE_PATH.glob("*.h5"):
        h5_file_paths.append(file)

print(h5_file_paths)

# opening and inspecting each image file
for path in h5_file_paths:
    with h5py.File(path, "r") as f:
        print(f"File: {path.name}")
        print("Keys in file:", list(f.keys()))
        infrared_data = f['Infrared']
        print(type(infrared_data))

        #  checking shape and dtype of the dataset
        if isinstance(infrared_data, h5py.Dataset):
            print("Shape:", infrared_data.shape)
            print("Data type:", infrared_data.dtype)

            data = infrared_data[:]
            print(data)

    # OpenCV resizing (note: cv2 expects shape as (width, height))
    # First normalize to 0–255 for resizing if dtype is float
    data_min = np.min(data)
    data_max = np.max(data)
    data_scaled = 255 * (data - data_min) / (data_max - data_min)
    data_scaled = data_scaled.astype(np.uint8)

    # Resize using cv2 (height, width order)
    # resized_array = cv2.resize(data_scaled, desired_size, interpolation=cv2.INTER_AREA)
    resized_array = cv2.resize(data_scaled, desired_size, interpolation=cv2.INTER_CUBIC)
    # Convert back to float normalized 0–1
    normalized = resized_array.astype(np.float32) / 255.0             # Image Normalization 


    #  Directory for the normalized level
    normalized_output_dir = os.path.join(os.getcwd(), "typhon_images", "processed", "normalized")
    os.makedirs(normalized_output_dir, exist_ok = True)
    # print(f"Saving normalized images to: {normalized_output_dir}")

    # Creating filename for saving the images
    filename = os.path.splitext(path.name)[0] + "_normalized.npy"
    normalized_img_path = os.path.join(normalized_output_dir, filename)

    # Saving the file
    np.save(normalized_img_path, normalized)

[WindowsPath('d:/Sneha/DA/DL_ntcc_project/typhon_images/image/197901/1978122906-197901-GMS1-1.h5'), WindowsPath('d:/Sneha/DA/DL_ntcc_project/typhon_images/image/197901/1978122909-197901-GMS1-1.h5'), WindowsPath('d:/Sneha/DA/DL_ntcc_project/typhon_images/image/197901/1978122912-197901-GMS1-1.h5'), WindowsPath('d:/Sneha/DA/DL_ntcc_project/typhon_images/image/197901/1978122916-197901-GMS1-1.h5'), WindowsPath('d:/Sneha/DA/DL_ntcc_project/typhon_images/image/197901/1978122918-197901-GMS1-1.h5'), WindowsPath('d:/Sneha/DA/DL_ntcc_project/typhon_images/image/197901/1978122921-197901-GMS1-1.h5'), WindowsPath('d:/Sneha/DA/DL_ntcc_project/typhon_images/image/197901/1978123000-197901-GMS1-1.h5'), WindowsPath('d:/Sneha/DA/DL_ntcc_project/typhon_images/image/197901/1978123003-197901-GMS1-1.h5'), WindowsPath('d:/Sneha/DA/DL_ntcc_project/typhon_images/image/197901/1978123006-197901-GMS1-1.h5'), WindowsPath('d:/Sneha/DA/DL_ntcc_project/typhon_images/image/197901/1978123009-197901-GMS1-1.h5'), WindowsPa

This is probably a thermal infrared image from a geostationary meteorological satellite (like GMS-1) — where:

Warmer areas (land, clouds) appear brighter (higher values).

Cooler areas (high clouds, ocean) appear darker (lower values).

### Visualizing the Normalized Images

In [ ]:
# Path to directory with normalized .npy files
npy_file_path = os.path.join(os.getcwd(), "typhon_images", "processed", "normalized")
npy_file = Path(npy_file_path)

# Loop through each .npy file and display it
for file in npy_file.glob("*.npy"):
    image = np.load(file)  # Load the actual array from the .npy file

    plt.imshow(image, cmap='grey')  # Note: correct spelling is 'gray'
    plt.colorbar(label="Normalized Intensity (0 to 1)")
    plt.title(f"Normalized Image: {file.name}")
    plt.show()

### **Data Labelling**

- Performed Using labelImg tool 
- Saved the 300 labeled images in a newly created folder named "xml_label_images"

Performing data augmentation on those 300 labeled images 

## **Data Augmenttaion**

In [ ]:
# Paths
image_dir = os.path.join(os.getcwd(), "label_images_png")
label_dir = os.path.join(os.getcwd(), "xml_label_images")
output_image_dir = "augmented_images"
output_label_dir = "augmented_labels"

# Create output folders if they don't exist
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Albumentations transform (you can customize)
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.7),
    A.RandomResizedCrop(size=(300, 300), scale=(0.8, 1.0), ratio=(0.75, 1.33), p=0.5)
], bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids']))

# Process each image
for img_path in glob(f"{image_dir}/*.png"):
    file_name = os.path.basename(img_path).split('.')[0]
    label_path = os.path.join(label_dir, file_name + ".txt")
    
    if not os.path.exists(label_path):
        print(f"[!] Label not found for {file_name}, skipping.")
        continue

    # Load image
    image = cv2.imread(img_path)
    height, width = image.shape[:2]

    # Load labels
    with open(label_path, 'r') as f:
        lines = f.readlines()

    bboxes = []
    category_ids = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue
        cls, x, y, w, h = map(float, parts)
        bboxes.append([x, y, w, h])
        category_ids.append(int(cls))

    # Perform 10 augmentations per image
    for i in range(10):
        try:
            augmented = transform(image=image, bboxes=bboxes, category_ids=category_ids)
            aug_img = augmented['image']
            aug_bboxes = augmented['bboxes']
            aug_ids = augmented['category_ids']

            # Save image
            aug_img_name = f"{file_name}_aug{i}.png"
            aug_img_path = os.path.join(output_image_dir, aug_img_name)
            cv2.imwrite(aug_img_path, aug_img)

            # Save updated label
            aug_label_path = os.path.join(output_label_dir, f"{file_name}_aug{i}.txt")
            with open(aug_label_path, 'w') as f:
                for bbox, cls in zip(aug_bboxes, aug_ids):
                    x, y, w, h = bbox
                    f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")
        except Exception as e:
            print(f"[!] Skipped {file_name}_aug{i} due to error: {e}")

print("Data augmentation completed! Check 'augmented_images' and 'augmented_labels'")
